In [1]:
import numpy as np
import random
import csv
import copy
import random
from datetime import datetime
import sys

limit_number = 15000
sys.setrecursionlimit(limit_number)

class TspProblem():
    def __init__(self):
        self.cities = []
        self.population = []
        self.total_cost = []
        self.newPop = []
        self.givenSolution = []
        
    def distance(self,x, y): # Euclidean distance
        dist = np.linalg.norm(np.array(x)-np.array(y))
        return dist
        
    def readTSP(self,file): # cities list
        with open(file, mode='r', newline='') as tsp:
            reader = csv.reader(tsp)
            for i in reader:
                self.cities.append(i)
    
    def readSol(self,file): # index of given cities list
        with open(file, mode='r', newline='') as solution:
            reader = csv.reader(solution)
            for i in reader:
                self.givenSolution.append(int(i[0]))

    def genPop(self,size): 
        for i in range(size):
            chromosome = self.givenSolution
            random.shuffle(chromosome)
            indexOfStart = chromosome.index(0) # find start point
            front = chromosome[indexOfStart:]
            back = chromosome[0:indexOfStart]
            chromosome = front + back
            chromosome.append(int(0))
            self.population.append(chromosome)  
        
    def fitnessEval(self):
        fitnessEval = []
        for i in range(len(self.population)):
            distance = [] 
            total_cost = 0
            for idx in range(len(self.givenSolution)):
                pos_city_1 = [float(self.cities[self.population[i][idx]][0]), float(self.cities[self.population[i][idx]][1])]
                pos_city_2 = [float(self.cities[self.population[i][idx+1]][0]), float(self.cities[self.population[i][idx+1]][1])]
                distance.append(self.distance(pos_city_1, pos_city_2)) 
            
            total_cost = sum(distance)
            fitnessEval.append(total_cost) # 총 거리
        return fitnessEval
    
    def select(self,fitnessEval): 
        parents = []
        modFit = []
        for i in fitnessEval:
            modFit.append((SIZEOFCHROMO * 100) - i)
        Idx = modFit.index(max(modFit))
        Idx2 = modFit.index(min(modFit))
        modFit[Idx] *= 2.0
        modFit[Idx2] *= 0.5
        max_value = sum(modFit)
        
        for j in range(2):
            pick = random.uniform(0, max_value)
            current = 0
            for i in modFit:
                current += i
                if current > pick:
                    parents.append(modFit.index(i))
                    break
        return parents
            
        
    def crossOver(self,selectIdx):
        child = [-1 for i in range(len(self.population[selectIdx[0]]))]
        ind = 0
        cross1 = (self.population[selectIdx[0]])
        cross2 = (self.population[selectIdx[1]])
        
        while True:
            # 자식의 값이 모두 채워지면 종료
            if child.count(-1)==0:
                break

            child[ind] = cross1[ind]
            ind = cross1.index(cross2[ind])
            
            if child[ind]!=-1 and child.count(-1)!=0:
                cross3 = cross1
                cross1 = cross2
                cross2 = cross3
                ind = child.index(-1)
        child.pop()
        child = self.mutate(child)
        child.append(child[0])
        
        return  self.newPop.append(child)  

    def findCloseNode(self,chromosome,point):
        distance = [] # distance 초기화
        for idx in range(len(chromosome)):
            pos_city_1 = [float(self.cities[chromosome[point]][0]), float(self.cities[chromosome[point]][1])]
            pos_city_2 = [float(self.cities[chromosome[idx]][0]), float(self.cities[chromosome[idx]][1])]
            
            distance.append(self.distance(pos_city_1, pos_city_2)) # 두 도시의 거리를 스택, 총 1000개가 입력된다.
            if distance[-1] == 0: distance[-1] = 100000
        nearestNode = distance.index(min(distance))
        return nearestNode
    
    def mutate(self,chromosome):
        if random.random() < MUTATION_RATE:
            for i in range(MUTATION_TIME):
                point1 = random.randint(1, len(chromosome)-2)
                nearestNode = self.findCloseNode(chromosome,point1)
                chromosome[point1+1], chromosome[nearestNode] = chromosome[nearestNode], chromosome[point1+1]
        return chromosome

    def changeGen(self):
        self.population = self.newPop.copy()
        self.newPop = []
    

  
f = open('/home/code/Jo/stdout2.txt', 'w')
SIZEOFCHROMO = 1000
INITPOPSIZE = 50
MUTATION_RATE = 0.1
MUTATION_TIME = 10
POPSIZE = 30
GENERATION = 10000
a = TspProblem()
a.readTSP('/home/code/Jo/TSP.csv')
a.readSol('/home/code/Jo/example_solution.csv')
a.genPop(INITPOPSIZE)                               
 
for i in range(GENERATION):
    fitness = a.fitnessEval()
    for j in range(POPSIZE):
        parentsIdx = a.select(fitness)
        a.crossOver(parentsIdx)

    a.changeGen()

    minimum = min(fitness)
    maximum = max(fitness)
    avg = sum(fitness)/len(fitness)
    print("{0}, {1}, {2}, {3}".format(i,minimum,maximum,avg))
    print("{0}, {1}, {2}, {3}".format(i,minimum,maximum,avg), file = f)
 
f.close()

sys.stdout = open('/home/code/Jo/stdout.txt', 'w')
print("{0}세대, 최소 비용은 {1} 최대 비용은 {2} 평균비용은 {3} ".format(i,minimum,maximum,avg))
fitIdx = fitness.index(min(fitness))
print(a.population[fitIdx])
sys.stdout.close()


0세대, 최소 비용은 50440.14637210166 최대 비용은 54315.58476849048 평균 비용은 52325.763190223835
1세대, 최소 비용은 50318.220309044955 최대 비용은 54216.262412300064 평균 비용은 52211.054707948584
2세대, 최소 비용은 50359.89732329978 최대 비용은 53739.684384665816 평균 비용은 52195.868445045955
3세대, 최소 비용은 50239.66831683107 최대 비용은 54652.55495403973 평균 비용은 52058.668628341795
4세대, 최소 비용은 50641.10396752725 최대 비용은 53528.11783684161 평균 비용은 52061.159822859525
5세대, 최소 비용은 50663.09765166174 최대 비용은 53694.665310975295 평균 비용은 52070.34628570412
6세대, 최소 비용은 50471.299381005265 최대 비용은 54307.02069208895 평균 비용은 52096.98689190153
7세대, 최소 비용은 50503.15591324501 최대 비용은 54278.15094178774 평균 비용은 52097.306128106015
8세대, 최소 비용은 49852.614412816714 최대 비용은 53894.73984304961 평균 비용은 52131.14777635857
9세대, 최소 비용은 49730.987997845165 최대 비용은 53981.322905914065 평균 비용은 52134.06407040477
10세대, 최소 비용은 50056.12000366014 최대 비용은 54910.33640473345 평균 비용은 52200.88508278239
11세대, 최소 비용은 50208.0792282386 최대 비용은 53866.53767274496 평균 비용은 52155.3088311117
12세대, 최소 비용은 50418.8136483

FileNotFoundError: [Errno 2] No such file or directory: '/home/code/Jo/stdout.txt'